https://leetcode.com/problems/my-calendar-iii/

In [ ]:
Implement a MyCalendarThree class to store your events. A new event can always be added.

Your class will have one method, book(int start, int end). Formally, this represents a booking on the half open interval [start, end), the range of real numbers x such that start <= x < end.

A K-booking happens when K events have some non-empty intersection (ie., there is some time that is common to all K events.)

For each call to the method MyCalendar.book, return an integer K representing the largest integer such that there exists a K-booking in the calendar.

Your class will be called like this: MyCalendarThree cal = new MyCalendarThree(); MyCalendarThree.book(start, end)
Example 1:

MyCalendarThree();
MyCalendarThree.book(10, 20); // returns 1
MyCalendarThree.book(50, 60); // returns 1
MyCalendarThree.book(10, 40); // returns 2
MyCalendarThree.book(5, 15); // returns 3
MyCalendarThree.book(5, 10); // returns 3
MyCalendarThree.book(25, 55); // returns 3
Explanation: 
The first two events can be booked and are disjoint, so the maximum K-booking is a 1-booking.
The third event [10, 40) intersects the first event, and the maximum K-booking is a 2-booking.
The remaining events cause the maximum K-booking to be only a 3-booking.
Note that the last event locally causes a 2-booking, but the answer is still 3 because
eg. [10, 20), [10, 40), and [5, 15) are still triple booked.
 

Note:

The number of calls to MyCalendarThree.book per test case will be at most 400.
In calls to MyCalendarThree.book(start, end), start and end are integers in the range [0, 10^9].


In [ ]:
# 中文
实现一个 MyCalendar 类来存放你的日程安排，你可以一直添加新的日程安排。

MyCalendar 有一个 book(int start, int end)方法。它意味着在start到end时间内增加一个日程安排，注意，这里的时间是半开区间，即 [start, end), 实数 x 的范围为，  start <= x < end。

当 K 个日程安排有一些时间上的交叉时（例如K个日程安排都在同一时间内），就会产生 K 次预订。

每次调用 MyCalendar.book方法时，返回一个整数 K ，表示最大的 K 次预订。

请按照以下步骤调用MyCalendar 类: MyCalendar cal = new MyCalendar(); MyCalendar.book(start, end)

[imos法](http://www.hankcs.com/program/algorithm/imos_method.html)介绍，[原文](http://imoz.jp/algorithms/imos_method.html)

imos法，或者叫图解法

思路：只统计出入店时刻的人数变化，入加一，出减一，最后累加每个时刻的统计量，得出其最大值

In [2]:
import collections
class MyCalendarThree(object):
    def __init__(self):
        self.book_list = collections.defaultdict(int)
    def book(self, start, end):
        self.book_list[start] += 1
        self.book_list[end] -= 1
        tmp = out = 0
        for k,v in sorted(self.book_list.items()):
            out += v
            if out > tmp:
                tmp = out
        return tmp

In [3]:
# 大神解法，暂时还看不懂。。。
class MyCalendarThree(object):
    def __init__(self):
        self.starts = []
        self.ends = []
        self.rooms = 0
    def book(self, start, end):
        import bisect
        # 先插入到对应的start 和 end
        bisect.insort(self.starts,start)
        bisect.insort(self.ends,end)
        # 获取当前左右的索引
        r = bisect.bisect_right(self.starts,end)
        l = bisect.bisect_left(self.ends,start)
        i,j = l,l
        res = 0
        while i < r and j < r:
            if self.starts[i] < self.ends[j]:
                res += 1
                i += 1
            else:
                i += 1
                j += 1
        self.rooms = max(self.rooms,res)
        return self.rooms

In [4]:
# 惰性传播
import collections
class MyCalendarThree(object):
    def __init__(self):
        # 节点值
        self.seg = collections.defaultdict(int)
        # 惰性树存储的当前节点值
        self.lazy = collections.defaultdict(int)
    def book(self, start, end):
        def update(s, e, l = 0, r = 10**9, ID = 1):
            if r <= s or e <= l: return 
            # 区间满足所给的start到end范围
            if s <= l < r <= e:
                self.seg[ID] += 1
                self.lazy[ID] += 1
            else:
                # 折半更新区间段
                m = (l + r) // 2
                update(s, e, l, m, 2 * ID)
                update(s, e, m, r, 2*ID+1)
                # 更新当前节点，通过惰性树中的值与左右孩子值进行更新。
                self.seg[ID] = self.lazy[ID] + max(self.seg[2*ID], self.seg[2*ID+1])
        update(start, end)
        return self.seg[1]

In [2]:
# 线段树，从下面一个博客中学来
class Node:
    def __init__(self, l, r, count):
        self.left = None # 左节点
        self.right = None # 右节点
        self.count = count
        self.m = -1 # -1代表被分割，其他数代表分割点
        self.l = l # [l,XX)
        self.r = r # [r, XX)
        
class MyCalendarThree(object):
    def __init__(self):
        self.root = Node(0, 10**9, 0)
        self.max = 0
        
    def book(self, start, end):
        self.add(start, end, self.root)
        return self.max
    
    def add(self, start, end, root):
        if root.m != -1: # 被分割过
            if end <= root.m: self.add(start, end, root.left) # end->m->r 则往左叶子方向
            elif start >= root.m: self.add(start, end, root.right) # l->m->start 则往右叶子方向
            else: # start->m->end 左右切分
                self.add(start, root.m, root.left)
                self.add(root.m, end, root.right)
            return
        if start == root.l and end == root.r:
            root.count += 1
            self.max = max(self.max, root.count)
        elif start == root.l:
            root.m = end
            root.left = Node(start, root.m, root.count+1)
            root.right = Node(root.m, root.r, root.count)
            self.max = max(self.max, root.count+1)
        elif end == root.r:
            root.m = start
            root.left = Node(root.l, root.m, root.count)
            root.right = Node(root.m, end, root.count+1)
            self.max = max(self.max, root.count+1)
        else:
            root.m = start
            root.left = Node(root.l, root.m, root.count)
            root.right = Node(root.m, root.r, root.count)
            self.add(start, end, root.right)

[<img src='images/732_My_Calendar_III_1.png'>](https://zxi.mytechroad.com/blog/geometry/732-my-calendar-iii/)